# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started. Once you have completed the code you can download the notebook for making a submission.

In [1]:
import numpy as np
import os
from imageio import imread
from skimage.transform import resize
import datetime
import os
import warnings
warnings.filterwarnings('ignore')


We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from tensorflow import keras
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

**data path: /home/datasets/Project_data**

In [3]:
train_doc = np.random.permutation(open('/home/datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/home/datasets/Project_data/val.csv').readlines())
batch_size = 32

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = np.array([1,2,4,6,8,10,12,14,16,18,20,22,24,26,28,29])
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t) // batch_size   # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    resized_image = resize(image, (100, 100, 3))
                    batch_data[folder, idx, :, :, :] = resized_image / 255
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
            if len(t) % batch_size != 0:
                remaining_data = len(t) % batch_size
                batch_data = np.zeros((remaining_data, len(img_idx), 100, 100, 3))
                batch_labels = np.zeros((remaining_data, 5))
            
                for folder in range(remaining_data):
                    folder_name = t[folder + (num_batches * batch_size)].split(';')[0]
                    imgs = os.listdir(os.path.join(source_path, folder_name))
                
                    for idx, item in enumerate(img_idx):
                        image = imread(os.path.join(source_path, folder_name, imgs[item])).astype(np.float32)
                        resized_image =  resize(image, (100, 100, 3))
                        batch_data[folder, idx, :, :, :] = resized_image / 255

                    batch_labels[folder, int(t[folder + (num_batches * batch_size)].strip().split(';')[2])] = 1

                yield batch_data, batch_labels  # Yield remaining data and labels

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [5]:
curr_dt_time = datetime.datetime.now()
train_path = '/home/datasets/Project_data/train'
val_path = '/home/datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 50 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 50


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, Flatten, BatchNormalization, Activation, Conv3D, MaxPooling3D, GlobalAveragePooling3D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers


#write your model here
model_3d = Sequential()

# 1st Conv3D layer
model_3d.add(Conv3D(filters=16, kernel_size=(3, 3, 3), input_shape=(16, 100, 100, 3), padding='same'))
model_3d.add(BatchNormalization())
model_3d.add(Activation('relu'))
model_3d.add(MaxPooling3D(pool_size=(2, 2, 2), padding='same'))

# 2nd Conv3D layer
model_3d.add(Conv3D(filters=32, kernel_size=(3, 3, 3), padding='same'))
model_3d.add(BatchNormalization())
model_3d.add(Activation('relu'))
model_3d.add(MaxPooling3D(pool_size=(2, 2, 2), padding='same'))

# 3rd Conv3D layer
model_3d.add(Conv3D(filters=64, kernel_size=(3, 3, 3), padding='same'))
model_3d.add(BatchNormalization())
model_3d.add(Activation('relu'))
model_3d.add(MaxPooling3D(pool_size=(2, 2, 2), padding='same'))


# Flatten and Dense layers
model_3d.add(Flatten())
model_3d.add(Dense(128))
model_3d.add(BatchNormalization())
model_3d.add(Activation('relu'))
model_3d.add(Dropout(0.5))

# Output layer
model_3d.add(Dense(5, activation='softmax'))

2024-07-30 18:21:27.081600: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1052] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-30 18:21:27.123969: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1052] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-30 18:21:27.124322: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1052] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-30 18:21:27.128524: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1052] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-07-30 18:21:27.128900: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1052] successful NUMA node read f

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [7]:
optimiser = "adam"
model_3d.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy','accuracy'])
print (model_3d.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 16, 100, 100, 16)  1312      
                                                                 
 batch_normalization (BatchN  (None, 16, 100, 100, 16)  64       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 16, 100, 100, 16)  0         
                                                                 
 max_pooling3d (MaxPooling3D  (None, 8, 50, 50, 16)    0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 8, 50, 50, 32)     13856     
                                                                 
 batch_normalization_1 (Batc  (None, 8, 50, 50, 32)    1

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [8]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [9]:
model_name = 'model_init_Conv_3D' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model_epoch-{epoch:05d}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto',save_freq = num_epochs)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, mode='auto', min_lr=0.00001)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit` method to decide the number of next() calls it need to make.

In [10]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [11]:
model_3d.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 32
Epoch 1/50


2024-07-30 18:21:31.695403: I tensorflow/stream_executor/cuda/cuda_dnn.cc:377] Loaded cuDNN version 8302
2024-07-30 18:21:33.350415: I tensorflow/stream_executor/cuda/cuda_blas.cc:1792] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


21/21 [==============================] - 46s 2s/step - loss: 1.0653 - categorical_accuracy: 0.6151 - accuracy: 0.6151 - val_loss: 1.7183 - val_categorical_accuracy: 0.4028 - val_accuracy: 0.4028 - lr: 0.0010
Epoch 2/50
21/21 [==============================] - 43s 2s/step - loss: 0.5966 - categorical_accuracy: 0.7801 - accuracy: 0.7801 - val_loss: 2.8507 - val_categorical_accuracy: 0.2361 - val_accuracy: 0.2361 - lr: 0.0010
Epoch 3/50
 7/21 [=========>....................] - ETA: 26s - loss: 0.5463 - categorical_accuracy: 0.8071 - accuracy: 0.8071
Epoch 00003: saving model to model_init_Conv_3D_2024-07-3018_21_26.958734/model_epoch-00003.h5
21/21 [==============================] - 45s 2s/step - loss: 0.4505 - categorical_accuracy: 0.8419 - accuracy: 0.8419 - val_loss: 3.2916 - val_categorical_accuracy: 0.3194 - val_accuracy: 0.3194 - lr: 0.0010
Epoch 4/50
21/21 [==============================] - ETA: 0s - loss: 0.3617 - categorical_accuracy: 0.8604 - accuracy: 0.8604
Epoch 00004: Reduce